<a href="https://colab.research.google.com/github/nigfuapp-web/Xoron-Dev/blob/beta/testac824c3ac0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Xoron-Dev Automation
This notebook clones the repository, installs dependencies, and executes the build script.

In [ ]:
%env PYDEVD_DISABLE_FILE_VALIDATION=1

In [ ]:
# 1. Clone the repository
!git clone -b beta --single-branch https://github.com/nigfuapp-web/Xoron-Dev.git

In [ ]:
# 2. Change directory into the project folder
%cd Xoron-Dev

In [ ]:
# 3. Install requirements
!pip install -r requirements.txt -q

In [ ]:
# Install requirement if missing
!pip install -q nvidia-ml-py

import threading
import time
import requests
from pynvml import *

def start_discord_monitor(webhook_url, interval=600):
    """
    Initializes a background thread to send GPU stats to Discord.
    """
    def monitor_loop():
        try:
            nvmlInit()
            device_count = nvmlDeviceGetCount()
        except Exception as e:
            print(f"❌ Could not initialize NVML: {e}")
            return

        while True:
            try:
                embed_fields = []
                for i in range(device_count):
                    handle = nvmlDeviceGetHandleByIndex(i)
                    info = nvmlDeviceGetMemoryInfo(handle)
                    util = nvmlDeviceGetUtilizationRates(handle)
                    name = nvmlDeviceGetName(handle)

                    vram_used = round(info.used / 1024**2, 2)
                    vram_total = round(info.total / 1024**2, 2)

                    embed_fields.append({
                        "name": f"Device {i}: {name}",
                        "value": f"**Load:** `{util.gpu}%` \n**VRAM:** `{vram_used} / {vram_total} MiB`",
                        "inline": True
                    })

                payload = {
                    "username": "Xoron-Dev Monitor",
                    "embeds": [{
                        "title": "📊 Kaggle Hardware Utilization",
                        "color": 5814783,
                        "fields": embed_fields,
                        "footer": {"text": f"Last Update: {time.strftime('%Y-%m-%d %H:%M:%S')}"}
                    }]
                }
                requests.post(webhook_url, json=payload)
            except Exception as e:
                print(f"⚠️ Monitor Loop Error: {e}")

            time.sleep(interval)

    # Launch daemon thread
    thread = threading.Thread(target=monitor_loop, daemon=True)
    thread.start()
    print(f"🚀 Discord monitor active (Polling every {interval}s)")

# --- CONFIGURATION ---
WEBHOOK = "https://discord.com/api/v10/webhooks/1469929340539113695/PoYLBti7TQ_lXHTGUw7Uz92IBu1L-EEzwLxRkVqoK-d4o7gYRTv_Xs7EsKJtJ25PW8SS"
start_discord_monitor(WEBHOOK, interval=600)


In [ ]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [ ]:
# 4. Execute the build script
%cd scripts

In [ ]:
!python build_unified_dataset.py